In [1]:
%matplotlib inline

/anaconda/envs/deep-learning/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/anaconda/envs/deep-learning/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
import numpy as np
import pandas as pd
import theano
import theano.tensor as T
import matplotlib.pyplot as plt
import keras 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder 
from keras.utils import np_utils
from sklearn.cross_validation import train_test_split
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation

Using TensorFlow backend.


For this section we will use the Kaggle otto challenge.
If you want to follow, Get the data from Kaggle: https://www.kaggle.com/c/otto-group-product-classification-challenge/data

#### About the data

The Otto Group is one of the world’s biggest e-commerce companies, A consistent analysis of the performance of products is crucial. However, due to diverse global infrastructure, many identical products get classified differently.
For this competition, we have provided a dataset with 93 features for more than 200,000 products. The objective is to build a predictive model which is able to distinguish between our main product categories. 
Each row corresponds to a single product. There are a total of 93 numerical features, which represent counts of different events. All features have been obfuscated and will not be defined any further.

https://www.kaggle.com/c/otto-group-product-classification-challenge/data

In [3]:
def load_data(path, train=True):
    """Load data from a CSV File
    
    Parameters
    ----------
    path: str
        The path to the CSV file
        
    train: bool (default True)
        Decide whether or not data are *training data*.
        If True, some random shuffling is applied.
        
    Return
    ------
    X: numpy.ndarray 
        The data as a multi dimensional array of floats
    ids: numpy.ndarray
        A vector of ids for each sample
    """
    df = pd.read_csv(path)
    X = df.values.copy()
    # print (X[0][1])
    
    for i in range(len(X)):
        for j in range(len(X[0])):
            X[i][j] = ord(X[i][j]) - 97
            if X[i][j] < 0:
                X[i][j] = 0
    
    if train:
        np.random.shuffle(X)  # https://youtu.be/uyUXoap67N8
        X, labels = X[:, 1:-1].astype(np.float32), X[:, -1]
        return X, labels
    else:
        X, ids = X[:, 1:].astype(np.float32), X[:, 0].astype(str)
        return X, ids

In [4]:
def preprocess_data(X, scaler=None):
    """Preprocess input data by standardise features 
    by removing the mean and scaling to unit variance"""
    if not scaler:
        scaler = StandardScaler()
        scaler.fit(X)
    X = scaler.transform(X)
    return X, scaler


def preprocess_labels(labels, encoder=None, categorical=True):
    """Encode labels with values among 0 and `n-classes-1`"""
    if not encoder:
        encoder = LabelEncoder()
        encoder.fit(labels)
    y = encoder.transform(labels).astype(np.int32)
    if categorical:
        y = np_utils.to_categorical(y)
    return y, encoder

In [5]:
print("Loading data...")

X, labels = load_data('/Users/HillOfFlame/TAVTech/ArtificialIntelligence/deep-learning-keras-tensorflow/data/mushrooms.csv', train=False)
X, scaler = preprocess_data(X)

Y, encoder = preprocess_labels(labels)
print("Training data loaded...")

# lst = [] 
# for i in range(20):
#     lst.append(Y[i])
# print("Y List:", lst)

# Divide Data into X, Y, and X_test, Y_test
# First, we will do so naively, in order to keep track. Later, will randomize.
                            # print ("Y.size:", Y.shape), print ("ind:", ind), print ("X.size:", X.shape)
size = Y.shape[0]
percentage = 4/float(5)
ind = int(size*percentage)

X, X_test = X[:ind], X[ind:]
Y, Y_test = Y[:ind], Y[ind:]


# No need to upload test data, since it is not contained in seperate file. Rather, better to randomly choose subsection of training data.
# X_test, ids = load_data('/Users/HillOfFlame/TAVTech/ArtificialIntelligence/deep-learning-keras-tensorflow/data/mushrooms.csv', train=False)
# X_test, ids = X_test[:1000], ids[:1000]
# print("Test data loaded...")


#Plotting the data
print(X[0])
print(X_test[0])
print("X.shape:", X.shape)
print("Y.shape:", Y.shape)

print("Scaler:", scaler)
X_test, _ = preprocess_data(X_test, scaler)


nb_classes = Y.shape[1]
print(nb_classes, 'classes')

dims = X.shape[1]
print(dims, 'dims')

Loading data...
Training data loaded...
[ 1.07775176  0.16938174  0.12010274  1.18593943  0.66338432  0.16289727
 -0.43884608  1.4946779  -0.1718898  -1.1448077   0.90143555  0.71536112
  0.63265646  0.75760239  0.76813     0.          0.15356779 -0.27837813
  0.90068269 -0.65194172 -0.15982252  2.20915222]
[-0.95089877  0.16938174 -1.08879435 -0.84324586 -1.02352381  0.16289727
 -0.43884608  1.4946779  -1.39874434  0.87351209 -0.57324481 -1.07227147
  0.63265646  0.75760239  0.76813     0.          0.15356779 -0.27837813
 -1.31834579  1.39299047  0.3246544  -0.88054681]
X.shape: (6499, 22)
Y.shape: (6499, 2)
Scaler: StandardScaler(copy=True, with_mean=True, with_std=True)
2 classes
22 dims


Now lets create and train a logistic regression model.

---

# Keras

## Deep Learning library for Theano and TensorFlow

Keras is a minimalist, highly modular neural networks library, written in Python and capable of running on top of either TensorFlow or Theano. It was developed with a focus on enabling fast experimentation. Being able to go from idea to result with the least possible delay is key to doing good research.
ref: https://keras.io/

## Why this name, Keras?

Keras (κέρας) means _horn_ in Greek. It is a reference to a literary image from ancient Greek and Latin literature, first found in the _Odyssey_, where dream spirits (_Oneiroi_, singular _Oneiros_) are divided between those who deceive men with false visions, who arrive to Earth through a gate of ivory, and those who announce a future that will come to pass, who arrive through a gate of horn. It's a play on the words κέρας (horn) / κραίνω (fulfill), and ἐλέφας (ivory) / ἐλεφαίρομαι (deceive).

Keras was initially developed as part of the research effort of project ONEIROS (Open-ended Neuro-Electronic Intelligent Robot Operating System).

>_"Oneiroi are beyond our unravelling --who can be sure what tale they tell? Not all that men look for comes to pass. Two gates there are that give passage to fleeting Oneiroi; one is made of horn, one of ivory. The Oneiroi that pass through sawn ivory are deceitful, bearing a message that will not be fulfilled; those that come out through polished horn have truth behind them, to be accomplished for men who see them."_ Homer, Odyssey 19. 562 ff (Shewring translation).

## Hands On - Keras Logistic Regression


In [6]:
"""
Create a massive function to find the ideal Density of the first layer.
"""

def optimize_NN (X, Y, i=30, j=5):
    
    dims = X.shape[1] #number of features in our data  - 22
    print(dims, 'dims')
    print("Building model...")

    nb_classes = Y.shape[1] #number of classes in our data - 2
    print(nb_classes, 'classes')

    model = Sequential() #intialize a model. This creates a new model.


    model.add(Dense(i, input_shape=(dims,)))
    model.add(Dense(j, input_shape=(dims,)))

    model.add(Dense(nb_classes))
    """
    Here, we are adding a fully connected network with 9 layers.
    The size of the input is 93. It's not an image. It is categorical data.
    Every time you add Dense, you are adding a new later.

    """

    model.add(Activation('softmax')) #add softmax activation
    """
    Softmax activation will 
    """


    model.compile(optimizer='sgd', loss='categorical_crossentropy') #add the sgd optimizer to optimize our model 
    """
    Optimizing using stochastic gradient descent.

    """


    model.fit(X, Y) #start optimizing 

    train_predictions = model.predict(X).argmax(axis=1)
    print (train_predictions)

    print('train accuracy is {0}'.format(sum(Y.argmax(axis=1)==train_predictions)/train_predictions.shape[0]))
    
    return (sum(Y.argmax(axis=1)==train_predictions)/train_predictions.shape[0])


i_range = [5,10,15,20,25,30,35]
j_range = [5,10,15,20,25,30,35]
d = {}

for i in i_range:
    for j in j_range:
        d[i,j] = optimize_NN (X, Y, i, j)
        
import operator
stats = {'a':1000, 'b':3000, 'c': 100}
best = max(d.items(), key=operator.itemgetter(1))[0]

print("Best Hyperparameters:", best)
    


22 dims
Building model...
2 classes
Epoch 1/10
6499/6499 [==============================] - 0s - loss: 0.5001     
Epoch 2/10
6499/6499 [==============================] - 0s - loss: 0.3160     
Epoch 3/10
6499/6499 [==============================] - 0s - loss: 0.2700     
Epoch 4/10
6499/6499 [==============================] - 0s - loss: 0.2439     
Epoch 5/10
6499/6499 [==============================] - 0s - loss: 0.2244     
Epoch 6/10
6499/6499 [==============================] - 0s - loss: 0.2096     
Epoch 7/10
6499/6499 [==============================] - 0s - loss: 0.1984     
Epoch 8/10
6499/6499 [==============================] - 0s - loss: 0.1896     
Epoch 9/10
6499/6499 [==============================] - 0s - loss: 0.1830     
Epoch 10/10
6499/6499 [==============================] - 0s - loss: 0.1777     
[0 1 1 ..., 0 0 0]
train accuracy is 0.9475303892906601
22 dims
Building model...
2 classes
Epoch 1/10
6499/6499 [==============================] - 0s - loss: 0.4346     
Ep

ValueError: I/O operation on closed file

In [ ]:
print (d)
# best = max(d.items(), key=operator.itemgetter(1))[0]
# print ("MAX:", max(d.items()))
print ("KEYS:", d.items())

In [ ]:
dims = X.shape[1] #number of features in our data  - 22
print(dims, 'dims')
print("Building model...")

nb_classes = Y.shape[1] #number of classes in our data - 2
print(nb_classes, 'classes')

model = Sequential() #intialize a model. This creates a new model.


model.add(Dense(30, input_shape=(dims,)))
model.add(Dense(5, input_shape=(dims,)))

model.add(Dense(nb_classes))
"""
Here, we are adding a fully connected network with 9 layers.
The size of the input is 93. It's not an image. It is categorical data.
Every time you add Dense, you are adding a new later.

"""

model.add(Activation('softmax')) #add softmax activation
"""
Softmax activation will 
"""


model.compile(optimizer='sgd', loss='categorical_crossentropy') #add the sgd optimizer to optimize our model 
"""
Optimizing using stochastic gradient descent.

"""


model.fit(X, Y) #start optimizing 

train_predictions = model.predict(X).argmax(axis=1)
print (train_predictions)

print('train accuracy is {0}'.format(sum(Y.argmax(axis=1)==train_predictions)/train_predictions.shape[0]))

In [ ]:
train_predictions = model.predict(X).argmax(axis=1)
print (train_predictions)

In [ ]:
print('train accuracy is {0}'.format(sum(Y.argmax(axis=1)==train_predictions)/train_predictions.shape[0]))

Simplicity is pretty impressive right? :)

Now lets understand:
<pre>The core data structure of Keras is a <b>model</b>, a way to organize layers. The main type of model is the <b>Sequential</b> model, a linear stack of layers.</pre>


What we did here is stacking a Fully Connected (<b>Dense</b>) layer of trainable weights from the input to the output and an <b>Activation</b> layer on top of the weights layer.

##### Dense

```python
from keras.layers.core import Dense

Dense(output_dim, init='glorot_uniform', activation='linear', 
      weights=None, W_regularizer=None, b_regularizer=None,
      activity_regularizer=None, W_constraint=None, 
      b_constraint=None, bias=True, input_dim=None)
```

##### Activation

```python
from keras.layers.core import Activation

Activation(activation)
```

##### Optimizer

If you need to, you can further configure your optimizer. A core principle of Keras is to make things reasonably simple, while allowing the user to be fully in control when they need to (the ultimate control being the easy extensibility of the source code).
Here we used <b>SGD</b> (stochastic gradient descent) as an optimization algorithm for our trainable weights.  

"Data Sciencing" this example a little bit more
=====

What we did here is nice, however in the real world it is not useable because of overfitting.
Lets try and solve it with cross validation.

##### Overfitting

In overfitting, a statistical model describes random error or noise instead of the underlying relationship. Overfitting occurs when a model is excessively complex, such as having too many parameters relative to the number of observations. 

A model that has been overfit has poor predictive performance, as it overreacts to minor fluctuations in the training data.


<img src ="imgs/overfitting.png">

<pre>To avoid overfitting, we will first split out data to training set and test set and test out model on the test set.
Next: we will use two of keras's callbacks <b>EarlyStopping</b> and <b>ModelCheckpoint</b></pre>

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.15, random_state=42)

fBestModel = 'best_model.h5' 
early_stop = EarlyStopping(monitor='val_loss', patience=4, verbose=1) 
best_model = ModelCheckpoint(fBestModel, verbose=0, save_best_only=True)
model.fit(X_train, Y_train, validation_data = (X_val, Y_val), nb_epoch=20, 
          batch_size=128, verbose=True, validation_split=0.15, 
          callbacks=[best_model, early_stop])

In [ ]:
train_predictions = model.predict(X_train).argmax(axis=1)
print('train accuracy is {0}'.format(sum(Y_train.argmax(axis=1)==train_predictions)/train_predictions.shape[0]))

In [ ]:
val_predictions = model.predict(X_val).argmax(axis=1)
print('test accuracy is {0}'.format(sum(Y_val.argmax(axis=1)==val_predictions)/val_predictions.shape[0]))

## Multi-Layer Perceptron and Fully Connected

So, how hard can it be to build a Multi-Layer percepton with keras? It is baiscly the same, just add more layers!

In [ ]:
my_model = Sequential()
my_model.add(Dense(70, input_shape=(dims,)))
my_model.add(Dense(nb_classes))
my_model.add(Activation('softmax'))

# my_model.add(Dense(100))
# my_model.add(Sigmoid())
# When you add an activation function, it applies only to the last layer that was written.
# my_model.add(Dense(512))
# my_model.add(Dense(10))

my_model.compile(optimizer='sgd', loss='categorical_crossentropy')
my_model.fit(X_train, Y_train, validation_data = (X_val, Y_val), nb_epoch=20, 
          batch_size=128, verbose=True, validation_split=0.15, 
          callbacks=[best_model, early_stop])

In [ ]:
model = Sequential()
model.add(Dense(100, input_shape=(dims,)))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))
model.compile(optimizer='sgd', loss='categorical_crossentropy')
model.fit(X_train, Y_train, validation_data = (X_val, Y_val), nb_epoch=20, 
          batch_size=128, verbose=True, validation_split=0.15, 
          callbacks=[best_model, early_stop])


In [ ]:
train_predictions = model.predict(X_train).argmax(axis=1)
print('train accuracy is {0}'.format(sum(Y_train.argmax(axis=1)==train_predictions)/train_predictions.shape[0]))

In [ ]:
val_predictions = model.predict(X_val).argmax(axis=1)
print('test accuracy is {0}'.format(sum(Y_val.argmax(axis=1)==val_predictions)/val_predictions.shape[0]))

Your Turn!

## Hands On - Keras Fully Connected


Take couple of minutes and Try and optimize the number of layers and the number of parameters in the layers to get the best results. 

In [ ]:
model = Sequential()
model.add(Dense(100, input_shape=(dims,)))
# Play with it! add as much layers as you want! try and get better results.
model.add(Dense(100))
model.add(Dense(512))
model.add(Dense(512))
model.add(Dense(100))
model.add(Dense(nb_classes))

model.add(Activation('softmax'))
model.compile(optimizer='sgd', loss='categorical_crossentropy')
model.fit(X_train, Y_train, validation_data = (X_val, Y_val), nb_epoch=50, 
          batch_size=128, verbose=True, validation_split=0.15, 
          callbacks=[best_model, early_stop]) 

In [ ]:
train_predictions = model.predict(X).argmax(axis=1)
print('train accuracy is {0}'.format(sum(Y.argmax(axis=1)==train_predictions)/train_predictions.shape[0]))

In [ ]:
val_predictions = model.predict(X_val).argmax(axis=1)
print('test accuracy is {0}'.format(sum(Y_val.argmax(axis=1)==val_predictions)/val_predictions.shape[0]))

### Let's overfit it

In [ ]:
model = Sequential()
model.add(Dense(512, input_shape=(dims,)))
# Play with it! add as much layers as you want! try and get better results.
model.add(Dense(512))
model.add(Dense(512))
model.add(Dense(512))
model.add(Dense(512))
model.add(Dense(512))
model.add(Dense(nb_classes))

model.add(Activation('softmax'))
model.compile(optimizer='sgd', loss='categorical_crossentropy')
history = model.fit(X_train, Y_train, validation_data = (X_val, Y_val), nb_epoch=30, 
          batch_size=128, verbose=True, validation_split=0.15, ) 

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

Building a question answering system, an image classification model, a Neural Turing Machine, a word2vec embedder or any other model is just as fast. The ideas behind deep learning are simple, so why should their implementation be painful?

#### Theoretical Motivations for depth

>Much has been studied about the depth of neural nets. Is has been proven mathematically[1] and empirically that convolutional neural network benifit from depth! 

[1] - On the Expressive Power of Deep Learning: A Tensor Analysis - Cohen, et al 2015

#### Theoretical Motivations for depth

One much quoted theorem about neural network states that:

>Universal approximation theorem states[1] that a feed-forward network with a single hidden layer containing a finite number of neurons (i.e., a multilayer perceptron), can approximate continuous functions on compact subsets of $\mathbb{R}^n$, under mild assumptions on the activation function. The theorem thus states that simple neural networks can represent a wide variety of interesting functions when given appropriate parameters; however, it does not touch upon the algorithmic learnability of those parameters.

[1] - Approximation Capabilities of Multilayer Feedforward Networks - Kurt Hornik 1991